In [14]:
from modules import Notion, Goodreads
import os

from dotenv import load_dotenv

load_dotenv()
NOTION_API_KEY = os.environ["NOTION_API_KEY"]
NOTION_DATABASE_ID = os.environ["NOTION_DATABASE_ID"]

# Provide the Goodreads URL to capture the metadata for
URL = "https://www.goodreads.com/book/show/296776.Life_Lessons"

goodreads = Goodreads(URL)

book = goodreads.get_metadata(verbose=True)

Name: Life Lessons: Two Experts on Death and Dying Teach Us About the Mysteries of Life and Living
Image: https://images-na.ssl-images-amazon.com/images/S/compressed.photo.goodreads.com/books/1378705391i/296776.jpg
Format: Paperback
Number of pages: 224
Language: English
ISBN: 9780684870755
Rating: 4.27
Number of ratings: 2081
Number of reviews: 243
Authors: ['Elisabeth Kübler-Ross', 'David Kessler']


In [15]:
notion = Notion(NOTION_API_KEY, NOTION_DATABASE_ID)
notion.post_book_from_goodreads(book)

<Response [200]>

In [11]:
book.name

'On Death and Dying'